<a href="https://colab.research.google.com/github/camilotayac/computational_chemistry-TC/blob/master/Colab/conversion_of_a_CG_model_to_an_all_atom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧙‍♂️  Conversion of a CG model to an all-atom ♾️  Conversión de un modelo CG en un modelo totalmente atómico

🐉 **@Tyac**

🏦 **Universidad de Caldas**

🌇 **Manizales - Colombia**



In [ ]:
# @title ## 🟢  Installation of programs and import of required libraries ♾️ Instalación de programas e importación de librerías necesarias
!echo y | pip uninstall dgl
!echo y | pip uninstall torch
!pip install dgl==0.9 torch==1.13.1
!pip install git+http://github.com/huhlim/cg2all
!pip install nglview==3.0.8
!pip install "ipywidgets>=7,<8"
!pip install simpletraj
from google.colab import output
output.enable_custom_widget_manager()
import nglview
import ipywidgets
import plotly.graph_objs as go
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import display, HTML
import simpletraj


Found existing installation: torch 2.3.0+cu121
Uninstalling torch-2.3.0+cu121:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/bin/torchrun
    /usr/local/lib/python3.10/dist-packages/functorch/*
    /usr/local/lib/python3.10/dist-packages/torch-2.3.0+cu121.dist-info/*
    /usr/local/lib/python3.10/dist-packages/torch/*
    /usr/local/lib/python3.10/dist-packages/torchgen/*
Proceed (Y/n)?   Successfully uninstalled torch-2.3.0+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 998.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 40.5 MB/s eta 0:00:00
ERROR

In [ ]:
# @title ## 📁 Files for analysis ♾️ Archivos para el análisis
# @markdown ### **🇺🇸** - Select whether you will work with tutorial files or with your own files
# @markdown ### **🇪🇸** - Seleccione si va a trabajar con archivos del tutorial o con archivos propios
files = 'Own files - Archivos propios'# @param ['Tutorial files - Archivos del tutorial', 'Own files - Archivos propios']
# @markdown #### 🔴 **🇺🇸 - Note:** add the address of your folder, example: ***folder_a*** or ***folder_a/folder_b***
# @markdown #### 🔴 **🇪🇸 - Nota:** agregue la dirección de su carpeta, ejemplo: ***carpeta_a*** o ***carpeta_a/carpeta_b***
drive_folder_path = "MD" #@param {type:"string"}

if files == 'Tutorial files - Archivos del tutorial':
  %cd /content
  !git clone https://github.com/camilotayac/computational_chemistry-TC.git
  %cd computational_chemistry-TC/Examples
elif files == 'Own files - Archivos propios':
  from google.colab import drive
  drive.mount('/content/drive')
  carpeta = "/content/drive/MyDrive/" + drive_folder_path
  %cd {carpeta}

ValueError: mount failed

In [ ]:
# @title ## 🟢 Variables for analysis ♾️ Variables para análisis

model_type = "Martini3" #@param ["CalphaBasedModel", "ResidueBasedModel", "SidechainModel", "CalphaCMModel", "CalphaSCModel", "BackboneModel", "MainchainModel", "Martini", "Martini3", "PRIMO"]

# @markdown ---

# @markdown ### **🇺🇸** - Analysis with or without track record
# @markdown ### **🇪🇸** - Análisis con o sin trayectoria



selection_trajectory = "with trajectory - con trayectoria"# @param ["with trajectory - con trayectoria", "without trajectory - sin trayectoria"]

# @markdown ---

# @markdown ### 🟣 without trajectory ♾️ sin trayectoria
# @markdown ### **🇺🇸** - Filename with extension **.pdb** o **.gro**
# @markdown ### **🇪🇸** - Nombre del archivo con extensión **.pdb** o **.gro**

file_name = "connected_traj.pdb" # @param {type:"string"}
file_name = file_name.split(" ")

# @markdown ---

# @markdown ### 🟣 with trajectory ♾️ con trayectoria


# @markdown ### **🇺🇸** - Filename with extension **.pdb** o **.gro**
# @markdown ### **🇪🇸** - Nombre del archivo con extensión **.pdb** o **.gro**

file_name_trajectory = "connected_traj.pdb" # @param {type:"string"}
file_name_trajectory = file_name_trajectory.split(" ")

# @markdown ### **🇺🇸** - Extension trajectory **.xtc** o **.trr**
# @markdown ### **🇪🇸** - Trayectoria con la extensión **.xtc**, **.dcd** o **.trr**

name_trajectory = "recentered_traj.xtc" # @param {type:"string"}
name_trajectory = name_trajectory.split(" ")

# @markdown ---

# @markdown #### 🔴 **🇺🇸 - Note:** for multiple files, separate them with ***spaces***, if an error occurs it may be due to a ***double space***
# @markdown #### 🔴 **🇪🇸 - Nota:** para varios archivos sepárelos con ***espacios***, si surge un error puede ser por un ***doble espacio***

# @markdown ---

if selection_trajectory == "without trajectory - sin trayectoria":
  views = []
  numero_filas = len(file_name)
  for files in file_name:
    name, extension = files.split(".")
    output = f"{name}_convert.{extension}"
    !convert_cg2all -p $files -o $output --cg $model_type
    view = nglview.show_structure_file(files)
    view.add_representation(repr_type='ball+stick')
    view._remote_call('setSize', target='Widget', args=['600px','600px'])
    view.camera='orthographic'
    views.append(view)
    view = nglview.show_structure_file(output)
    view.add_representation(repr_type='ball+stick')
    view._remote_call('setSize', target='Widget', args=['600px','600px'])
    view.camera='orthographic'
    views.append(view)

elif selection_trajectory == "with trajectory - con trayectoria":
  views = []
  numero_filas = len(file_name_trajectory)
  contador = 0
  for files in file_name_trajectory:
    name, extension = files.split(".")
    name2, extension2 = name_trajectory[contador].split(".")
    output = f"{name}_convert.{extension}"
    !convert_cg2all -p $files -o $output --cg $model_type
    input = name_trajectory[contador]
    output_atom=f"{name}_trajectory_new.{extension2}"
    !convert_cg2all -p $files -d $input -o $output_atom --cg $model_type
    view = nglview.show_simpletraj(nglview.SimpletrajTrajectory(name_trajectory[contador], files), gui=True)
    view.clear_representations()
    view.add_representation(repr_type='ball+stick')
    view._remote_call('setSize', target='Widget', args=['600px','600px'])
    view.camera='orthographic'
    views.append(view)
    view = nglview.show_simpletraj(nglview.SimpletrajTrajectory(output_atom,output ), gui=True)
    view.clear_representations()
    view.add_representation(repr_type='ball+stick')
    view._remote_call('setSize', target='Widget', args=['600px','600px'])
    view.camera='orthographic'
    views.append(view)

ipywidgets.GridBox(views, layout=ipywidgets.Layout(grid_template_columns=f"repeat(2, 0fr)", grid_template_rows=f"repeat({numero_filas}, 0fr)"))


{'loading_ckpt': 0.40734195709228516, 'model_configuration': 0.1831984519958496, 'loading_input': 2.0594162940979004, 'forward_pass': 43.20039200782776, 'writing_output': 2.6601648330688477, 'total': 48.51051354408264}
100% 51/51 [37:02<00:00, 43.59s/it]
{'loading_ckpt': 0.32100605964660645, 'model_configuration': 0.15325665473937988, 'loading_input': 2.5330846309661865, 'forward_pass': 2221.4584488868713, 'writing_output': 2.701748847961426, 'total': 2227.167545080185}


GridBox(children=(NGLWidget(max_frame=50), NGLWidget(max_frame=50)), layout=Layout(grid_template_columns='repe…

---

## 🟢 bibliography ♾️ Bibliografía
---

### **🇺🇸**
This Colab was based on the tutorial [cg2all](https://github.com/huhlim/cg2all?tab=readme-ov-file). Please contact cg2all for quotation

---

### **🇪🇸**
Este Colab se basó en el tutorial [cg2all](https://github.com/huhlim/cg2all?tab=readme-ov-file). Por favor dirigirse a cg2all para citar